In [1]:
import pandas as pd
import numpy as np

In [2]:
ingredient_data = "dataset/ingredient.csv"
recipe_data =  "dataset/recipe.csv"
ingredient_df = pd.read_csv(ingredient_data)
recipe_df = pd.read_csv(recipe_data)

In [3]:
recipe_df.head()

,query,recipe_id,publisher,source_url,image_url,social_rank,publisher_url,title,sum_cal,sum_fat,sum_carb,sum_protein
0,carrot,35169,Closet Cooking,http://www.closetcooking.com/2011/11/buffalo-c...,http://forkify-api.herokuapp.com/images/Buffal...,100.0,http://closetcooking.com,Buffalo Chicken Chowder,1716.504410,36.712325,169.449600,172.055435
1,carrot,35171,Closet Cooking,http://www.closetcooking.com/2011/08/buffalo-c...,http://forkify-api.herokuapp.com/images/Buffal...,100.0,http://closetcooking.com,Buffalo Chicken Grilled Cheese Sandwich,136.561000,10.121800,5.510300,5.949400
2,carrot,47893,101 Cookbooks,http://www.101cookbooks.com/archives/kale-mark...,http://forkify-api.herokuapp.com/images/kale_m...,100.0,http://www.101cookbooks.com,Kale Market Salad,1396.555755,93.653767,133.624733,24.786457
3,carrot,36259,Simply Recipes,http://www.simplyrecipes.com/recipes/easy_shep...,http://forkify-api.herokuapp.com/images/shephe...,100.0,http://simplyrecipes.com,Easy Shepherd&#8217;s Pie,2568.342575,132.299872,133.432454,194.369595
4,carrot,47233,The Pioneer Woman,http://thepioneerwoman.com/cooking/2009/05/rya...,http://forkify-api.herokuapp.com/images/354427...,100.0,http://thepioneerwoman.com,Pastor Ryan’s Bolognese Sauce,3894.514009,301.983685,111.800097,165.475420


In [4]:
ingredient_df.head()

,query,recipe_id,quantity,unit,weight_g,ingredient,nutrition_key,nutrition_value,cal,fat,cah,protein,original ingredient
0,carrot,35169,2.0,tablespoons,8.60000,butter,butter,"{'cal': 7.17, 'fat': 0.81, 'carb': 0.001, 'pro...",61.66200,6.966000,0.0086,0.068800,2 tablespoons butter
1,carrot,35169,1.0,pound,453.59237,chicken,chicken,"{'cal': 1.65, 'fat': 0.036, 'carb': 0.0, 'prot...",748.42741,16.329325,0.0000,140.613635,"1 pound chicken, cut into bite sized pieces"
2,carrot,35169,1.0,NaN,142.00000,onion,onions,"{'cal': 0.4, 'fat': 0.001, 'carb': 0.09, 'prot...",56.80000,0.142000,12.7800,1.562000,"1 onion, diced"
3,carrot,35169,2.0,NaN,99.00000,carrots,carrot,"{'cal': 0.41, 'fat': 0.002, 'carb': 0.1, 'prot...",40.59000,0.198000,9.9000,0.891000,"2 carrots, diced"
4,carrot,35169,2.0,stalks,60.00000,celery,celery,"{'cal': 0.16, 'fat': 0.002, 'carb': 0.034, 'pr...",9.60000,0.120000,2.0400,0.420000,"2 stalks celery, diced"


## Method1: Not considering ingredient quantity 

### Preprocessing

In [5]:
# generate map {'recipe_id': [ingredient1,ingredient2... ]}
recipe_map = {}
for ind, row in ingredient_df.iterrows(): 
    if row['recipe_id'] in recipe_map.keys():
        recipe_map.get(row['recipe_id']).append(row['ingredient'])
    else:
        recipe_map[row['recipe_id']] = [row['ingredient']]

In [6]:
recipe_ing_df = pd.DataFrame(data = recipe_map.items(),columns=['recipe_id','ingredients'])

In [7]:
recipe_ing_df.head()

,recipe_id,ingredients
0,35169,"[butter, chicken, onion, carrots, celery, garl..."
1,35171,"[cooked shredded chicken, hot sauce, mayo, car..."
2,47893,"[Green Garlic Dressing:, green garlic, fine gr..."
3,36259,"[ground round beef, onion chopped, vegetables ..."
4,47233,"[Olive Oil, Grated Carrots, Large Red Onion, G..."


In [8]:
import re
import ast
import string
import unidecode

from nltk.stem import WordNetLemmatizer

def ingredient_parser(ingreds):
    '''
    
    This function takes in a list (but it is a string as it comes from pandas dataframe) of 
       ingredients and performs some preprocessing. 
       For example:
       input = '['1 x 1.6kg whole duck', '2 heaped teaspoons Chinese five-spice powder', '1 clementine',
                 '6 fresh bay leaves', 'GRAVY', '', '1 bulb of garlic', '2 carrots', '2 red onions', 
                 '3 tablespoons plain flour', '100 ml Marsala', '1 litre organic chicken stock']'
       
       output = ['duck', 'chinese five spice powder', 'clementine', 'fresh bay leaf', 'gravy', 'garlic',
                 'carrot', 'red onion', 'plain flour', 'marsala', 'organic chicken stock']
    '''
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.', 'fluid ounce', 'fl oz', 'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter', 'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']
    words_to_remove = ['fresh', 'oil', 'a', 'red', 'bunch', 'and', 'clove', 'or', 'leaf', 'chilli', 'large', 'extra', 'sprig', 'ground', 'handful', 'free', 'small', 'pepper', 'virgin', 'range', 'from', 'dried', 'sustainable', 'black', 'peeled', 'higher', 'welfare', 'seed', 'for', 'finely', 'freshly', 'sea', 'quality', 'white', 'ripe', 'few', 'piece', 'source', 'to', 'organic', 'flat', 'smoked', 'ginger', 'sliced', 'green', 'picked', 'the', 'stick', 'plain', 'plus', 'mixed', 'mint', 'bay', 'basil', 'your', 'cumin', 'optional', 'fennel', 'serve', 'mustard', 'unsalted', 'baby', 'paprika', 'fat', 'ask', 'natural', 'skin', 'roughly', 'into', 'such', 'cut', 'good', 'brown', 'grated', 'trimmed', 'oregano', 'powder', 'yellow', 'dusting', 'knob', 'frozen', 'on', 'deseeded', 'low', 'runny', 'balsamic', 'cooked', 'streaky', 'nutmeg', 'sage', 'rasher', 'zest', 'pin', 'groundnut', 'breadcrumb', 'turmeric', 'halved', 'grating', 'stalk', 'light', 'tinned', 'dry', 'soft', 'rocket', 'bone', 'colour', 'washed', 'skinless', 'leftover', 'splash', 'removed', 'dijon', 'thick', 'big', 'hot', 'drained', 'sized', 'chestnut', 'watercress', 'fishmonger', 'english', 'dill', 'caper', 'raw', 'worcestershire', 'flake', 'cider', 'cayenne', 'tbsp', 'leg', 'pine', 'wild', 'if', 'fine', 'herb', 'almond', 'shoulder', 'cube', 'dressing', 'with', 'chunk', 'spice', 'thumb', 'garam', 'new', 'little', 'punnet', 'peppercorn', 'shelled', 'saffron', 'other''chopped', 'salt', 'olive', 'taste', 'can', 'sauce', 'water', 'diced', 'package', 'italian', 'shredded', 'divided', 'parsley', 'vinegar', 'all', 'purpose', 'crushed', 'juice', 'more', 'coriander', 'bell', 'needed', 'thinly', 'boneless', 'half', 'thyme', 'cubed', 'cinnamon', 'cilantro', 'jar', 'seasoning', 'rosemary', 'extract', 'sweet', 'baking', 'beaten', 'heavy', 'seeded', 'tin', 'vanilla', 'uncooked', 'crumb', 'style', 'thin', 'nut', 'coarsely', 'spring', 'chili', 'cornstarch', 'strip', 'cardamom', 'rinsed', 'honey', 'cherry', 'root', 'quartered', 'head', 'softened', 'container', 'crumbled', 'frying', 'lean', 'cooking', 'roasted', 'warm', 'whipping', 'thawed', 'corn', 'pitted', 'sun', 'kosher', 'bite', 'toasted', 'lasagna', 'split', 'melted', 'degree', 'lengthwise', 'romano', 'packed', 'pod', 'anchovy', 'rom', 'prepared', 'juiced', 'fluid', 'floret', 'room', 'active', 'seasoned', 'mix', 'deveined', 'lightly', 'anise', 'thai', 'size', 'unsweetened', 'torn', 'wedge', 'sour', 'basmati', 'marinara', 'dark', 'temperature', 'garnish', 'bouillon', 'loaf', 'shell', 'reggiano', 'canola', 'parmigiano', 'round', 'canned', 'ghee', 'crust', 'long', 'broken', 'ketchup', 'bulk', 'cleaned', 'condensed', 'sherry', 'provolone', 'cold', 'soda', 'cottage', 'spray', 'tamarind', 'pecorino', 'shortening', 'part', 'bottle', 'sodium', 'cocoa', 'grain', 'french', 'roast', 'stem', 'link', 'firm', 'asafoetida', 'mild', 'dash', 'boiling']
    # The ingredient list is now a string so we need to turn it back into a list. We use ast.literal_eval
    if isinstance(ingreds, list):
        ingredients = ingreds
    else:
        ingredients = ast.literal_eval(ingreds)
    # We first get rid of all the punctuation. We make use of str.maketrans. It takes three input 
    # arguments 'x', 'y', 'z'. 'x' and 'y' must be equal-length strings and characters in 'x'
    # are replaced by characters in 'y'. 'z' is a string (string.punctuation here) where each character
    #  in the string is mapped to None. 
    translator = str.maketrans('', '', string.punctuation)
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
        i.translate(translator)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items] #''.join((c for c in unicodedata.normalize('NFD', items) if unicodedata.category(c) != 'Mn'))
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
            ingred_list.append(' '.join(items)) 
    ingred_list = " ".join(ingred_list)
    return ingred_list

In [9]:
recipe_ing_df['ingredients_parsed'] = recipe_ing_df['ingredients'].apply(lambda x: ingredient_parser(x))
recipe_ing_df

,recipe_id,ingredients,ingredients_parsed
0,35169,"[butter, chicken, onion, carrots, celery, garl...",butter chicken onion carrot celery garlic flou...
1,35171,"[cooked shredded chicken, hot sauce, mayo, car...",chicken mayo carrot celery onion blue cheese c...
2,47893,"[Green Garlic Dressing:, green garlic, fine gr...",garlic garlic lemon avocado kale farro wheat b...
3,36259,"[ground round beef, onion chopped, vegetables ...",beef onion chopped vegetable chopped carrot po...
4,47233,"[Olive Oil, Grated Carrots, Large Red Onion, G...",carrot onion beef tomato paste garlic wine tom...
...,...,...,...
2251,50331,"[beef short ribs, butter, celery stalks, large...",beef short rib butter celery carrot onion wine...
2252,54683,"[extra-virgin olive oil, medium onion, garlic ...",onion garlic carrot celery rib chicken stock b...
2253,055eb1,"[boneless beef chuck, olive oil, carrots, cele...",beef chuck carrot celery rib onion garlic toma...
2254,f69f3e,"[bottled clam juice, littleneck clams, flour, ...",bottled clam littleneck clam flour shallot rib...


In [10]:
from gensim.models import Word2Vec

# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.ingredients_parsed:
        split_doc = doc.split()
        split_doc.sort()
        corpus_sorted.append(split_doc)
    return corpus_sorted
  
# calculate average length of each document 
def get_window(corpus):
    lengths = [len(doc) for doc in corpus]
    avg_len = float(sum(lengths)) / len(lengths)
    print("window size: "+ str(round(avg_len)))
    return round(avg_len)

In [11]:
# get corpus
corpus = get_and_sort_corpus(recipe_ing_df)
print(f"Length of corpus: {len(corpus)}")
# train and save CBOW Word2Vec model
model_cbow = Word2Vec(
#   corpus, sg=0, workers=8, window=get_window(corpus), min_count=1, vector_size=100
  corpus, sg=0, workers=8, window=10, min_count=1, vector_size=100

)
model_cbow.save('models/model_cbow.bin')
print("Word2Vec model successfully trained")

Length of corpus: 2256
Word2Vec model successfully trained


In [12]:
word_vectors = model_cbow.wv

In [13]:
word_vectors.most_similar("cheese")

[('cheddar', 0.9677807092666626),
 ('chive', 0.9369904398918152),
 ('bread', 0.9252020716667175),
 ('colander', 0.9237316250801086),
 ('additional', 0.920989453792572),
 ('artichoke', 0.9190930724143982),
 ('breast', 0.9180366396903992),
 ('celery', 0.9176560640335083),
 ('blue', 0.9066484570503235),
 ('chile', 0.9060304164886475)]

### Ingredient Embedding

In [14]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, model_cbow):

        self.model_cbow = model_cbow
        self.word_idf_weight = None
        self.vector_size = model_cbow.wv.vector_size

    def fit(self, docs): 
        """
        Build a tfidf model to compute each word's idf as its weight.
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  
        # if a word was never seen it is given idf of the max of known idf value
        max_idf = max(tfidf.idf_)  
        self.word_idf_weight = defaultdict(
            lambda: max_idf,
            [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()],
        )
        return self

    def transform(self, docs): 
        doc_word_vector = self.doc_average_list(docs)
        return doc_word_vector

    def doc_average(self, doc):
        """
        Compute weighted mean of documents word embeddings
        """

        mean = []
        for word in doc:
            if word in self.model_cbow.wv.index_to_key:
                mean.append(
                    self.model_cbow.wv.get_vector(word) * self.word_idf_weight[word]
                ) 

        if not mean:  
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def doc_average_list(self, docs):
        return np.vstack([self.doc_average(doc) for doc in docs])

### Recommendation

In [15]:
recipe_ing_df

,recipe_id,ingredients,ingredients_parsed
0,35169,"[butter, chicken, onion, carrots, celery, garl...",butter chicken onion carrot celery garlic flou...
1,35171,"[cooked shredded chicken, hot sauce, mayo, car...",chicken mayo carrot celery onion blue cheese c...
2,47893,"[Green Garlic Dressing:, green garlic, fine gr...",garlic garlic lemon avocado kale farro wheat b...
3,36259,"[ground round beef, onion chopped, vegetables ...",beef onion chopped vegetable chopped carrot po...
4,47233,"[Olive Oil, Grated Carrots, Large Red Onion, G...",carrot onion beef tomato paste garlic wine tom...
...,...,...,...
2251,50331,"[beef short ribs, butter, celery stalks, large...",beef short rib butter celery carrot onion wine...
2252,54683,"[extra-virgin olive oil, medium onion, garlic ...",onion garlic carrot celery rib chicken stock b...
2253,055eb1,"[boneless beef chuck, olive oil, carrots, cele...",beef chuck carrot celery rib onion garlic toma...
2254,f69f3e,"[bottled clam juice, littleneck clams, flour, ...",bottled clam littleneck clam flour shallot rib...


In [16]:
recipe_df

,query,recipe_id,publisher,source_url,image_url,social_rank,publisher_url,title,sum_cal,sum_fat,sum_carb,sum_protein
0,carrot,35169,Closet Cooking,http://www.closetcooking.com/2011/11/buffalo-c...,http://forkify-api.herokuapp.com/images/Buffal...,100.000000,http://closetcooking.com,Buffalo Chicken Chowder,1716.504410,36.712325,169.449600,172.055435
1,carrot,35171,Closet Cooking,http://www.closetcooking.com/2011/08/buffalo-c...,http://forkify-api.herokuapp.com/images/Buffal...,100.000000,http://closetcooking.com,Buffalo Chicken Grilled Cheese Sandwich,136.561000,10.121800,5.510300,5.949400
2,carrot,47893,101 Cookbooks,http://www.101cookbooks.com/archives/kale-mark...,http://forkify-api.herokuapp.com/images/kale_m...,100.000000,http://www.101cookbooks.com,Kale Market Salad,1396.555755,93.653767,133.624733,24.786457
3,carrot,36259,Simply Recipes,http://www.simplyrecipes.com/recipes/easy_shep...,http://forkify-api.herokuapp.com/images/shephe...,100.000000,http://simplyrecipes.com,Easy Shepherd&#8217;s Pie,2568.342575,132.299872,133.432454,194.369595
4,carrot,47233,The Pioneer Woman,http://thepioneerwoman.com/cooking/2009/05/rya...,http://forkify-api.herokuapp.com/images/354427...,100.000000,http://thepioneerwoman.com,Pastor Ryan’s Bolognese Sauce,3894.514009,301.983685,111.800097,165.475420
...,...,...,...,...,...,...,...,...,...,...,...,...
3161,ribs,54683,Two Peas and Their Pod,http://www.twopeasandtheirpod.com/easy-chicken...,http://forkify-api.herokuapp.com/images/436959...,99.998908,http://www.twopeasandtheirpod.com,Easy Chicken and Rice Soup,534.752000,7.830000,92.484000,31.867000
3162,ribs,055eb1,Epicurious,http://www.epicurious.com/recipes/food/views/B...,http://forkify-api.herokuapp.com/images/350935...,99.996798,http://www.epicurious.com,Beef Stew with Potatoes and Carrots,8639.499911,384.312336,444.198146,664.491830
3163,ribs,37010,Simply Recipes,http://www.simplyrecipes.com/recipes/steak_faj...,http://forkify-api.herokuapp.com/images/steakf...,99.996666,http://simplyrecipes.com,Steak Fajitas,1557.788623,100.627050,49.601000,122.178292
3164,ribs,f69f3e,Epicurious,http://www.epicurious.com/recipes/food/views/M...,http://forkify-api.herokuapp.com/images/511574...,99.995892,http://www.epicurious.com,Manhattan Clam Chowder,4887.468138,161.480437,232.260054,598.298872


In [17]:
recipe_ing_df.loc[recipe_ing_df['recipe_id'] == '1519']

,recipe_id,ingredients,ingredients_parsed


In [18]:
recipe_ing_df['recipe_id'][1519]

'b66bb2'

In [19]:
def get_topN_recommendations(N, scores):
    """
    Rank scores and output a pandas data frame containing all the details of the top N recipes.
    :param scores: list of cosine similarities
    """
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]

    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["score","recipe", "ingredients", "sum_cal","sum_fat","sum_carb","sum_protein", "url"])
    count = 0
    print(top)
    for i in top: # i is row index for score (recipe_ing_df)
        recipe_id = recipe_ing_df['recipe_id'][i]
        recommendation.at[count, "score"] = round(scores[i],5)
        recommendation.at[count, "ingredients"] = recipe_ing_df['ingredients'][i]       
        
        recipe_row_ind = recipe_df.loc[recipe_df['recipe_id'] == recipe_id].index[0]
        recommendation.at[count, "recipe"] = recipe_df["title"][recipe_row_ind]
        recommendation.at[count, "url"] = recipe_df["source_url"][recipe_row_ind]
        recommendation.at[count, "sum_cal"] = recipe_df["sum_cal"][recipe_row_ind]
        recommendation.at[count, "sum_carb"] = recipe_df["sum_carb"][recipe_row_ind]
        recommendation.at[count, "sum_protein"] = recipe_df["sum_protein"][recipe_row_ind]
        
        count += 1
    return recommendation

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

def get_recs(ingredients, N=5):
    """
    Get the top N recipe recomendations.
    :param ingredients: comma seperated string listing ingredients
    :param N: number of recommendations
    :param mean: False if using tfidf weighted embeddings, True if using simple mean
    """
    # load in word2vec model
#     model = Word2Vec.load("models/model_cbow.bin")
    model = model_cbow
    # normalize embeddings
    model.init_sims(replace=True)
    if model:
        print("Successfully loaded model")


    # use TF-IDF as weights for each word embedding
    tfidf_vec_tr = TfidfEmbeddingVectorizer(model)
    tfidf_vec_tr.fit(corpus)
    doc_vec = tfidf_vec_tr.transform(corpus)
    doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
    assert len(doc_vec) == len(corpus)

        
    # create embeddings for input text
    input = ingredients
    # create tokens with elements
    input = input.split(",")
    # parse ingredient list
    input = ingredient_parser(input)
    # get embeddings for ingredient doc

    input_embedding = tfidf_vec_tr.transform([input])[0].reshape(1, -1)
    print(input_embedding.shape)

    # get cosine similarity between input embedding and all the document embeddings
    cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
    scores = list(cos_sim)
    # Filter top N recommendations
    recommendations = get_topN_recommendations(N, scores)
    return recommendations

In [21]:
recipe_ing_df[1050:1060]

,recipe_id,ingredients,ingredients_parsed
1050,c36a54,"[cubed honeydew melon, cubed cantaloupe, thinl...",honeydew melon cantaloupe lemon prosciutto ins...
1051,34096e,"[1"" cubes peeled honeydew melon, fresh lime ju...",honeydew melon lime sugar
1052,7c1c37,[cubed seedless watermelon (from a 4 pound mel...,seedless watermelon melon sugar lime
1053,59515b,"[double cream, caster sugar, very ripe charent...",double cream caster sugar very charentais melo...
1054,49052,"[thinly sliced pancetta, orecchiette, Kosher s...",pancetta orecchiette champagne wine melon chop...
1055,49118,"[sugar, mint plus small leaves, thinly sliced ...",sugar chile bean lime melon coconut lime
1056,22516,"[cantaloupe - peeled, honeydew melon - peeled,...",cantaloupe honeydew melon lime sugar
1057,38054,"[orange juice, honey, extra-virgin olive oil, ...",orange soy chinese five pork tenderloin pineap...
1058,49178,"[chopped cantaloupe, fresh lemon juice, loosel...",chopped cantaloupe lemon loosely
1059,614f94,"[raw wheat berries, raw amaranth, heavy cream,...",wheat berry amaranth cream sugar sugar yogurt ...


In [22]:
user_input = "honeydew, melon, lime, sugar"
get_recs(user_input)

C:\Users\Zhao Lutong\.conda\envs\MR\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  app.launch_new_instance()


Successfully loaded model
(1, 100)
[1519, 854, 1384, 1712, 1343]


,score,recipe,ingredients,sum_cal,sum_fat,sum_carb,sum_protein,url
0,0.99643,Cadbury Creme Egg Cupcakes,"[paper cupcake liners, Batter for 24 cupcakes....",618.735,NaN,95.792,3.072,http://www.mybakingaddiction.com/cadbury-creme...
1,0.99615,Cookie Monster cupcakes,"[Cupcakes, Frosting, The frosting I made using...",2000.837,NaN,145.636,63.37,http://www.bbcgoodfood.com/recipes/873655/cook...
2,0.99594,Herbed Buttermilk Popcorn,[-Documented increased herbicide use on GMO cr...,282.012,NaN,57.621,12.394,http://www.101cookbooks.com/archives/000131.html
3,0.99527,Creating your wedding cake,"[x top tier, x middle tier, x bottom tier, hal...",13748.278,NaN,1618.992,79.177,http://www.bbcgoodfood.com/recipes/4578/creati...
4,0.99512,English-Muffin Egg Pizzas,"[English muffins, olive oil]",753.755,NaN,0.458,0.078,http://www.realsimple.com/food-recipes/browse-...
